# Goal: Develop Kalman filter and smoother using Cython and compare to numpy implementation 

In [1]:
import Cython
import cython
import scipy
import pandas as pd
import numpy as np

%load_ext cython

Current issues
- Memoryviews become both C-contigious and F-contigious after running Kalman iteration -> depends on way of slicing and shape of array

- A contigious array is an array whose elements are stored in adjecant memory block. A C-contigious array is an array whose adjacent elements are in rows together, whereas F-contigious is the Columns, so opposite what you'd expect based on the first letters
- Can only ommit trailing arguments in cdef functions. If cpdef, need the gil because python arguments were returned. Simply changing the order of arguments fixed like 10 errors in one step... All the errors required the gil. 

In [2]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm, ddot, dgemv, daxpy, dcopy, dgetri 
from libcpp cimport bool

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double cddot(int N, double[::1] x, int incX, double[::1] y, int incY) nogil:
    """
    
    custom dot product
        
        arguments
            n: length of the vectors x and y
        
        returns: type double
        
    """
    return ddot(&N, &x[0], &incX, &y[0], &incY)

    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double c_xAx(double[::1, :] A, double[::1] x, double[::1] Ax, double[::1] xAx, int incX=1, int inc_Ax=1, double alpha=1.0, double beta=0.0) nogil:
    """
    
    custom matrix vector x'Ax
    
        output: xAx
        
        returns: type double (can do arithmetic in C)
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &Ax[0], &inc_Ax)
    return ddot(&M, &x[0], &incX, &Ax[0], &inc_Ax)





Error compiling Cython file:
------------------------------------------------------------
...

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm, ddot, dgemv, daxpy, dcopy, dgetri 
^
------------------------------------------------------------

/Users/nielsota/.ipython/cython/_cython_magic_6c25fadd00fd3e8d290d7e5e06d5a601.pyx:5:0: 'scipy/linalg/cython_blas/dgetri.pxd' not found


In [282]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm, ddot, dgemv, daxpy, dcopy, daxpy
from scipy.linalg.cython_lapack cimport dgetri
from libcpp cimport bool

@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_vcopy(double[::1] x, double[::1] y, int N, int incX=1, int incY=1) nogil:
    """
    
    custom copy
        
        arguments
            n: vectors x and y, copies x into y
        
        returns: void
        
    """
    
    dcopy(&N, &x[0], &incX, &y[0], &incY)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_vec_add(double[::1] x, double[::1] y, double alpha=1.0, double beta=0.0, int incX=1, int incY=1) nogil:
    """
    
    custom vector addition: y = alpha * x + y
    
        output: stored in y
        
    """
    
    cdef int N = x.shape[0]
    
    daxpy(&N, &alpha, &x[0], &incX, &y[0], &incY)
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cdef void cAx(double[::1, :] A, double[::1] x, double[::1] y, double alpha=1.0, double beta=0.0, int incX=1, int incY=1) nogil:
    """
    
    custom matrix vector alpha * Ax + beta y
    
        output: stored in y
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &y[0], &incY)
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cdef void cATx(double[::1, :] A, double[::1] x, double[::1] y, double alpha=1.0, double beta=0.0, int incX=1, int incY=1) nogil:
    """
    
    custom matrix vector alpha * Ax + beta y
    
        output: stored in y
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    dgemv('T', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &y[0], &incY)
    

@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_MM(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3, double alpha = 1.0, double beta = 0.0) nogil:
    """
    
    Matrix matrix multiplication M3 = alpha * M1 * M2 + beta * M3
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
   
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    #with gil:
        #print(f'First matrix shape: {np.asarray(M1).shape}')
        #print(f'Second matrix shape: {np.asarray(M2).shape}')
        #print(f'Output matrix shape: {np.asarray(M3).shape}')
        #print(f'K: {K}')
        #print(f'M2.shape[0]: {M2.shape[0]}')
        #print()
        #if int(K) != int(M2.shape[0]):
        #    raise ValueError('dimension mismatch')
    
    
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'N', &M, &N, &K, &alpha, &M1[0,0], &M,
                                             &M2[0,0], &K, &beta,
                                             &M3[0,0], &M,)
    #with gil:
    #    print(np.asarray(M3))

@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_MTM(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3, double alpha = 1.0, double beta = 0.0) nogil:
    """
    
    Matrix matrix multiplication M3 = alpha * M1.T * M2 + beta * M3
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
   
    # switch for the transposed version
    cdef int M = M1.shape[1]
    cdef int K = M1.shape[0]
    cdef int N = M2.shape[1]
    
    #with gil:
        #print(f'First matrix shape: {np.asarray(M1).T.shape}')
        #print(f'Second matrix shape: {np.asarray(M2).shape}')
        #print(f'Output matrix shape: {np.asarray(M3).shape}')
        #print()
        #if M != M2.shape[0]:
        #    raise ValueError('dimension mismatch')
    
    
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('T', 'N', &M, &N, &K, &alpha, &M1[0,0], &K,
                                             &M2[0,0], &K, &beta,
                                             &M3[0,0], &M,)
    #with gil:
    #    print(np.asarray(M3))

    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_MMT(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3, double alpha = 1.0, double beta = 0.0) nogil:
    """
    
    Matrix matrix multiplication M3 = alpha * M1 * M2.T + beta * M3
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
    
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    
    # try to use the shape of the transposed matrix
    cdef int N = M2.shape[0]
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'T', &M, &N, &K, &alpha, &M1[0,0], &M,
                                        &M2[0,0], &N, &beta,
                                        &M3[0,0], &M,)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_inv(double[::1, :] A, int[::1] IPIV, double[::1, :] WORK, int info, int LWORK) nogil:
    
    """
    
    Matrix inversion
    
        returns: void, but result stored in A
        
    """
    
    
    cdef int N = A.shape[0]
    #cdef int info = 0
    #cdef int LWORK = 1

    #cdef int[::1] IPIV  = np.zeros((N,), order='f', dtype=np.intc)
    #cdef double[::1, :] WORK = np.zeros((1, 1), order='f')

    dgetri(&N, &A[0, 0], &N, &IPIV[0], &WORK[0,0], &LWORK, &info)
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_kalman_iteration(double[::1, :] Z, double[::1, :] T, 
                              double[::1, :] R, double[::1, :] Q, 
                              double[::1] a, 
                              double[::1] y, double[::1] v,
                              double[::1, :] P, double[::1, :] H,
                              double[::1, :] F, double[::1, :] F_inv, 
                              double[::1, :] I_s, double[::1, :] I_p,  
                              double[::1, :] AB, double[::1, :] PxP_aux,
                              double[::1, :] M,  double[::1, :] K,
                              double[::1] att, double[::1, :] Ptt,
                              double[::1] a_next, double[::1, :] P_next) nogil:
    """
    
    Kalman iteration
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
    
    cdef int info = 0
    cdef int LWORK = 1
    with gil:
        IPIV  = np.zeros((F.shape[0],), order='f', dtype=np.intc)
        WORK = np.zeros((1, 1), order='f')

    #### compute v = y + Za
    
    # copy y into v, this is where v will be stored
    c_vcopy(y, v, 2)
    
    # v -> v - Za = y + alpha * Za because of copy line above
    cATx(Z, a, v , alpha= -1.0)
    
    #### compute F = ZPZ' + H
    
    # F_ = ZP + 0F = ZP
    #with gil:
    #    print(np.asarray(Z))
    #    print(np.asarray(P))
    #    print(np.asarray(AB))
    c_MM(Z, P, AB)
    
    
    # F. = F_Z + 0*F = ZPZ'
    c_MMT(Z, AB, F)
    #with gil:
        #print(np.asarray(F))
    
    # F = F. + H = ZPZ' + H
    c_MM(I_s, H, F, alpha = 1.0, beta = 1.0)
    #with gil:
        #print(np.asarray(F))
    
    #### compute F inverse
    
    # copy F into F_inv
    c_MM(F, I_s, F_inv)
    
    # compute inverse of F
    c_inv(F_inv, IPIV, WORK, info, LWORK)
    
    #### compute M and K
    
    # M = PZ'F_inv
    c_MMT(P, Z, AB)
    c_MTM(AB, F_inv, M)
    
    # K = TM
    c_MM(T, M, K)
    
    #### compute att and Ptt
    
    # att
    cAx(M, v, att)
    c_vec_add(a, att)
    
    # Ptt
    c_MM(M, F, AB)
    c_MM(M, AB, Ptt)
    c_MM(P, I_p, Ptt, alpha=1.0, beta = -1.0)
    
    
    #### compute a_next and P_next
    cAx(T, att, a_next)
    
    # P_next
    c_MM(T, Ptt, PxP_aux)
    c_MMT(PxP_aux, T, P_next)
    c_MM(R, Q, PxP_aux)
    c_MMT(PxP_aux, R, P_next, alpha = 1.0, beta = 1.0)


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_kalman_filter(double[::1, :, :] Z, double[::1, :, :] T, 
                              double[::1, :, :] R, double[::1, :, :] Q, 
                              double[::1, :] a, 
                              double[::1, :] y, double[::1, :] v,
                              double[::1, :, :] P, double[::1, :, :] H,
                              double[::1, :, :] F, double[::1, :, :] F_inv, 
                              double[::1, :, :] I_s, double[::1, :, :] I_p,  
                              double[::1, :, :] AB, double[::1, :, :] PxP_aux,
                              double[::1, :, :] M,  double[::1, :, :] K,
                              double[::1, :] att, double[::1, :, :] Ptt,
                              double[::1, :] a_next, double[::1, :, :] P_next,
                              int Time, int [:] t_next) nogil:
    """
    
    Kalman iteration
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
    
    #.copy_fortran()
    
    #with gil:
        #print(np.asarray(P[:, :, 0]))
        #print(np.asarray(P[:, :, 0]).flags)
        #print(np.asarray(P[:, :, 0]).strides)
        
        #print(np.asarray(T[:, :, 0]))
        #print(np.asarray(T[:, :, 0]).flags)
        
        #print(np.asarray(R[:, :, 0]))
        #print(np.asarray(R[:, :, 0]).flags)
        
        #print(np.asarray(a[:, 0]))
        #print(np.asarray(y[:, 0]).flags)
    
    #for t in range(0, Time):
    #    c_kalman_iteration(Z[:, :, t], T[:, :, t], R[:, :, t], Q[:, :, t], 
    #                a[:, t], y[:, t], v[:, t], P[:, :, t], 
    #                H[:, :, t], F[:, :, t], F_inv[:, :, t], 
    #                I_s[:, :, t], I_p[:, :, t], AB[:, :, t], PxP_aux[:, :, t], 
    #                M[:, :, t], K[:, :, t], 
    #                att[:, t], Ptt[:, :, t], a_next[:, t], P_next[:, :, t])
    
    for t in range(0, Time):
        c_kalman_iteration(Z[:, :, t], T[:, :, t], R[:, :, t], Q[:, :, t], 
                    a[:, t], y[:, t], v[:, t], P[:, :, t], 
                    H[:, :, t], F[:, :, t], F_inv[:, :, t], 
                    I_s[:, :, t], I_p[:, :, t], AB[:, :, t], PxP_aux[:, :, t], 
                    M[:, :, t], K[:, :, t], 
                    att[:, t], Ptt[:, :, t], a[:, t_next[t]], P[:, :, t_next[t]])
    
    
    #c_kalman_iteration(Z[:, :, 0], T[:, :, 0], R[:, :, 0], Q[:, :, 0], 
    #                a[:, 0], y[:, 0], v[:, 0], P[:, :, 0], 
    #                H[:, :, 0], F[:, :, 0], F_inv[:, :, 0], 
    #                I_s[:, :, 0], I_p[:, :, 0], AB[:, :, 0], PxP_aux[:, :, 0], 
    #                M[:, :, 0], K[:, :, 0], 
    #                att[:, 0], Ptt[:, :, 0], a_next[:, 0], P_next[:, :, 0])
    

In file included from /Users/nielsota/.ipython/cython/_cython_magic_0120cdff8069fb02c012f02b70567bb3.c:812:
In file included from /Users/nielsota/.pyenv/versions/3.10.1/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /Users/nielsota/.pyenv/versions/3.10.1/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/nielsota/.pyenv/versions/3.10.1/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1948:
/Users/nielsota/.pyenv/versions/3.10.1/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.


## Test Single Iteration

In [247]:
# system variables
a = np.zeros((2,), order='f') + 1
Z = np.zeros((1, 2), order='f') + 1
R = np.zeros((2, 1), order='f') + 1
Q = np.zeros((1, 1), order='f') + 1
T = np.zeros((2, 2), order='f') + 1
v = np.zeros((1,), order='f')
y = np.zeros((1,), order='f') + 1
P = np.eye(2, order='f')
H = np.zeros((1,1), order='f') + 1
F = np.zeros((1,1), order='f')
F_inv = np.zeros((1,1), order='f')

# same shape as a, one gain per state element
M = np.zeros((2,1), order='f') + 1
K = np.zeros((2,1), order='f') + 1

# helper variables
Is = np.eye(1, order='f')
Ip = np.eye(2, order='f')
ZP = np.zeros((Z.shape[0], Z.shape[1]), order='f') 
PxP_aux = np.eye(2, order='f')

# state variables
att = np.zeros((2,), order='f') + 1
Ptt = np.eye(2, order='f')
a_next = np.zeros((2,), order='f') + 1
P_next = np.eye(2, order='f')

In [248]:
print(f'a: {a} with shape: {a.shape}')
print(f'Z: {Z} with shape: {Z.shape}') 
print(f'v: {v} with shape: {v.shape}')
print(f'y: {y} with shape: {y.shape}')
print(f'P: {P} with shape: {P.shape}')
print(f'H: {H} with shape: {H.shape}') 
print(f'F: {F} with shape: {F.shape}')
print(f'F_inv: {F_inv} with shape: {F_inv.shape}')

print(f'ZP: {ZP} with shape: {ZP.shape}')
print()

print(f'T: {T} with shape: {T.shape}') 
print(f'R: {R} with shape: {R.shape}') 
print(f'Q: {Q} with shape: {Q.shape}') 

a: [1. 1.] with shape: (2,)
Z: [[1. 1.]] with shape: (1, 2)
v: [0.] with shape: (1,)
y: [1.] with shape: (1,)
P: [[1. 0.]
 [0. 1.]] with shape: (2, 2)
H: [[1.]] with shape: (1, 1)
F: [[0.]] with shape: (1, 1)
F_inv: [[0.]] with shape: (1, 1)
ZP: [[0. 0.]] with shape: (1, 2)

T: [[1. 1.]
 [1. 1.]] with shape: (2, 2)
R: [[1.]
 [1.]] with shape: (2, 1)
Q: [[1.]] with shape: (1, 1)


In [249]:
c_kalman_iteration(Z, T, R, Q, a, y, v, P, H, F, F_inv, Is, Ip, ZP, PxP_aux, M, K, att, Ptt, a_next, P_next)

In [250]:
print(f'a: {a}')
print(f'Z: {Z}') 
print(f'v: {v}')
print(f'y: {y}')
print(f'P: {P}')
print(f'H: {H}') 
print(f'F: {F}')
print(f'F_inv: {F_inv}')
print(f'ZP: {ZP}')
print(f'I: {Is}')
print(f'M: {M}')
print(f'K: {K}')
print(f'att: {att}')
print(f'Ptt: {Ptt}')
print(f'a_next: {a_next}')
print()
print(f'P_next: {P_next}')

a: [1. 1.]
Z: [[1. 1.]]
v: [-1.]
y: [1.]
P: [[1. 0.]
 [0. 1.]]
H: [[1.]]
F: [[3.]]
F_inv: [[0.33333333]]
ZP: [[1. 1.]]
I: [[1.]]
M: [[0.33333333]
 [0.33333333]]
K: [[0.66666667]
 [0.66666667]]
att: [0.66666667 0.66666667]
Ptt: [[ 0.66666667 -0.33333333]
 [-0.33333333  0.66666667]]
a_next: [1.33333333 1.33333333]

P_next: [[1.66666667 1.66666667]
 [1.66666667 1.66666667]]


In [251]:
a.strides

(8,)

In [252]:
a.strides

(8,)

## Test Filter

In [253]:
# system variables
Time = 100

a = np.zeros((2, Time), order='f') + 1
Z = np.zeros((1, 2, Time), order='f') + 1
R = np.zeros((2, 1, Time), order='f') + 1
Q = np.zeros((1, 1, Time), order='f') + 1
T = np.zeros((2, 2, Time), order='f') + 1
v = np.zeros((1, Time), order='f')
y = np.zeros((1, Time), order='f') + 1
P = np.eye(2, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')

H = np.zeros((1,1, Time), order='f') + 1
F = np.zeros((1,1, Time), order='f')
F_inv = np.zeros((1,1, Time), order='f')

# same shape as a, one gain per state element
M = np.zeros((2,1, Time), order='f') + 1
K = np.zeros((2,1, Time), order='f') + 1

# helper variables
Is = np.eye(1, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')
Ip = np.eye(2, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')
ZP = np.zeros((Z.shape[0], Z.shape[1], Time), order='f') 
PxP_aux = np.eye(2, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')

# state variables
att = np.zeros((2, Time), order='f') + 1
Ptt = np.eye(2, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')
a_next = np.zeros((2, Time), order='f') + 1
P_next = np.eye(2, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')

In [301]:
Time = int(100)
t_next = np.arange(Time, dtype=np.int32) + int(1)
t_next

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int32)

In [302]:
P[:, :, 0].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [303]:
c_kalman_filter(Z, T, R, Q, a, y, v, P, H, F, F_inv, Is, Ip, ZP, PxP_aux, M, K, att, Ptt, a_next, P_next, Time, t_next)

In [309]:
t = 99

print(f'a: {a[:, t]}')
print(f'Z: {Z[:, :, t]}') 
print(f'v: {v[:, t]}')
print(f'y: {y[:, t]}')
print(f'P: {P[:, :, t]}')
print(f'H: {H[:, :, t]}') 
print(f'F: {F[:, :, t]}')
print(f'F_inv: {F_inv[:, :, t]}')
print(f'ZP: {ZP[:, :, t]}')
print(f'M: {M[:, :, t]}')
print(f'K: {K[:, :, t]}')
print(f'att: {att[:, t]}')
print(f'Ptt: {Ptt[:, :, t]}')
print(f'a_next: {a_next[:, t]}')
print()
print(f'P_next: {P_next[:, :, t]}')

a: [10713.38949027 10713.38949027]
Z: [[1. 1.]]
v: [-10713.38949027]
y: [1.]
P: [[2.35610723 2.35610723]
 [2.35610723 2.35610723]]
H: [[1.]]
F: [[10.4244289]]
F_inv: [[0.09592852]]
ZP: [[4.71221445 4.71221445]]
M: [[0.45203574]
 [0.45203574]]
K: [[0.90407148]
 [0.90407148]]
att: [5870.55452547 5870.55452547]
Ptt: [[0.22601787 0.22601787]
 [0.22601787 0.22601787]]
a_next: [1.33333333 1.33333333]

P_next: [[1.66666667 1.66666667]
 [1.66666667 1.66666667]]


# Local Level Trend

In [310]:
# system variables
Time = 100

a = np.zeros((1, Time), order='f') + 1
Z = np.zeros((1, 1, Time), order='f') + 1
R = np.zeros((1, 1, Time), order='f') + 1
Q = np.zeros((1, 1, Time), order='f') + 1
T = np.zeros((1, 1, Time), order='f') + 1
v = np.zeros((1, Time), order='f')
y = np.zeros((1, Time), order='f') + 1
P = np.eye(1, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')

H = np.zeros((1,1, Time), order='f') + 1
F = np.zeros((1,1, Time), order='f')
F_inv = np.zeros((1,1, Time), order='f')

# same shape as a, one gain per state element
M = np.zeros((1,1, Time), order='f') + 1
K = np.zeros((1,1, Time), order='f') + 1

# helper variables
Is = np.eye(1, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')
Ip = np.eye(1, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')
ZP = np.zeros((Z.shape[0], Z.shape[1], Time), order='f') 
PxP_aux = np.eye(1, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')

# state variables
att = np.zeros((1, Time), order='f') + 1
Ptt = np.eye(1, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')
a_next = np.zeros((1, Time), order='f') + 1
P_next = np.eye(1, order='f')[:, :, None].repeat(Time, axis=2).copy(order='f')

In [313]:
c_kalman_filter(Z, T, R, Q, a, y, v, P, H, F, F_inv, Is, Ip, ZP, PxP_aux, M, K, att, Ptt, a_next, P_next, Time, t_next)

In [314]:
a

array([[1.00000000e+00, 5.00000000e-01, 2.00000000e-01, 7.69230769e-02,
        2.94117647e-02, 1.12359551e-02, 4.29184549e-03, 1.63934426e-03,
        6.26174076e-04, 2.39177230e-04, 9.13575735e-05, 3.48954880e-05,
        1.33288904e-05, 5.09118309e-06, 1.94465890e-06, 7.42793602e-07,
        2.83721909e-07, 1.08372126e-07, 4.13944687e-08, 1.58112801e-08,
        6.03937159e-09, 2.30683468e-09, 8.81132441e-10, 3.36562644e-10,
        1.28555491e-10, 4.91038280e-11, 1.87559933e-11, 7.16415195e-12,
        2.73646254e-12, 1.04523568e-12, 3.99244505e-13, 1.52497831e-13,
        5.82489882e-14, 2.22491337e-14, 8.49841285e-15, 3.24610486e-15,
        1.23990172e-15, 4.73600316e-16, 1.80899224e-16, 6.90973549e-17,
        2.63928410e-17, 1.00811682e-17, 3.85066361e-18, 1.47082262e-18,
        5.61804250e-19, 2.14590128e-19, 8.19661354e-20, 3.13082778e-20,
        1.19586980e-20, 4.56781617e-21, 1.74475052e-21, 6.66435397e-22,
        2.54555671e-22, 9.72316141e-23, 3.71391718e-23, 1.418590